<h3> Beauty of SQL RANK Function | SQL Interview Question and Answers | Covid Cases </h3>

In [0]:
%sql
create table covid(city varchar(50),days date,cases int);
delete from covid;
insert into covid values('DELHI','2022-01-01',100);
insert into covid values('DELHI','2022-01-02',200);
insert into covid values('DELHI','2022-01-03',300);

insert into covid values('MUMBAI','2022-01-01',100);
insert into covid values('MUMBAI','2022-01-02',100);
insert into covid values('MUMBAI','2022-01-03',300);

insert into covid values('CHENNAI','2022-01-01',100);
insert into covid values('CHENNAI','2022-01-02',200);
insert into covid values('CHENNAI','2022-01-03',150);

insert into covid values('BANGALORE','2022-01-01',100);
insert into covid values('BANGALORE','2022-01-02',300);
insert into covid values('BANGALORE','2022-01-03',200);
insert into covid values('BANGALORE','2022-01-04',400);

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from covid

city,days,cases
BANGALORE,2022-01-01,100
BANGALORE,2022-01-02,300
BANGALORE,2022-01-03,200
BANGALORE,2022-01-04,400
CHENNAI,2022-01-01,100
CHENNAI,2022-01-02,200
CHENNAI,2022-01-03,150
MUMBAI,2022-01-02,100
MUMBAI,2022-01-03,300
MUMBAI,2022-01-01,100


In [0]:
%sql

select city from (
select *,rank()over(partition by city order by days)days_rnk,rank()over(partition by city order by cases)cases_rnk
,rank()over(partition by city order by days)-rank()over(partition by city order by cases) as diff
 from covid)a1
 group by city
 having count(distinct diff)==1 and avg(diff)=0

city
DELHI


<h3> solution in pyspark <\h3>

In [0]:
from pyspark.sql.functions import col,rank,countDistinct,avg
from pyspark.sql.window import Window

In [0]:
df_c=spark.sql("select * from covid")
df_c.show()

+---------+----------+-----+
|     city|      days|cases|
+---------+----------+-----+
|BANGALORE|2022-01-01|  100|
|BANGALORE|2022-01-02|  300|
|BANGALORE|2022-01-03|  200|
|BANGALORE|2022-01-04|  400|
|  CHENNAI|2022-01-01|  100|
|  CHENNAI|2022-01-02|  200|
|  CHENNAI|2022-01-03|  150|
|   MUMBAI|2022-01-02|  100|
|   MUMBAI|2022-01-03|  300|
|   MUMBAI|2022-01-01|  100|
|    DELHI|2022-01-02|  200|
|    DELHI|2022-01-01|  100|
|    DELHI|2022-01-03|  300|
+---------+----------+-----+



In [0]:
df_c=df_c.withColumn("day_rnk",rank().over(Window.partitionBy("city").orderBy("days"))).\
    withColumn("case_rnk",rank().over(Window.partitionBy("city").orderBy("cases")))
df_f=df_c.withColumn("diff",col("day_rnk")-col("case_rnk")).groupBy("city")\
    .agg(countDistinct("diff").alias("c_dist"),avg("diff").alias("avg_diff"))
df_f.filter((col("c_dist")==1)&(col("avg_diff")==0.0)).select("city").show()

+-----+
| city|
+-----+
|DELHI|
+-----+

